In [54]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rayleigh

In [55]:
def convert_ITIs_to_phase_angles(ITIs):
    phase_angles = []
    for i in range(1, len(ITIs)):
        # Calculate phase angle for each ITI
        phase_angle = (ITIs[i] / ITIs[i-1]) * 360  # Convert to degrees
        phase_angles.append(phase_angle)
    return phase_angles

def calculate_R_bar(phase_angles):
    # Convert phase angles to radians
    phase_angles_radians = np.deg2rad(phase_angles)
    
    # Calculate mean resultant length (R-bar)
    R_bar = np.abs(np.mean(np.exp(1j * phase_angles_radians)))
    
    return R_bar

# Example usage:
ITIs = [0.4, 1, 1.6, 2.4, 3.0]  # Example ITI values in milliseconds
phase_angles = convert_ITIs_to_phase_angles(ITIs)
R_bar = calculate_R_bar(phase_angles)

print("Phase Angles:", phase_angles)
print("R-bar (Stability Measure):", R_bar)


Phase Angles: [900.0, 576.0, 539.9999999999999, 450.0]
R-bar (Stability Measure): 0.7097753814110334


In [56]:
def replace_outliers_iqr(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    original_data = data.copy()

    # Replace outliers with the nearest bound
    data[data < lower_bound] = lower_bound
    data[data > upper_bound] = upper_bound
    
    # plt.figure(figsize=(5, 5))
    # plt.plot(original_data, label='Original Data', color='blue', marker='o', linestyle='--')
    # plt.plot(data, label='Filtered Data', color='red', marker='o', linestyle='--')
    # plt.title('Original vs Filtered Data')
    # plt.xlabel('Index')
    # plt.ylabel('Value')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    return data

# Example usage:
# time_series = np.array([1, 2, 3, 10, 5, 6, 7, 8, 9, 10])
# filtered_series = remove_outliers_iqr(time_series)
# print("Original time series:", time_series)
# print("Filtered time series without outliers:", filtered_series)


In [57]:
# Load tapping data from a file with moving average
def load_tapping_data(file_path):
    tapping_times = None
    with open(file_path, "r") as file:
        tapping_times = file.read()
    time_points = []
    for line in tapping_times.split('\n'):
        if line.strip():
            time_points.append(float(line.split()[0]))

    # select only after 45 seconds
    time_points = [time for time in time_points if time > 45]

    tapping_intervals = np.diff(time_points)
    tapping_intervals = replace_outliers_iqr(tapping_intervals)

    return tapping_intervals

In [58]:
task_ids = ['A','B','C','D']
# participant, the overlayee
bpms = {'A':[109,109], 'B':[122,109], 'C':[122,51], 'D':[51,122]}

source_file_path = '/home/anika/Desktop/MMT/data_handling/duet/Ujjwal_files_used.txt'

# Read the text file
with open('duet/Ujjwal_files_used.txt', 'r') as file:
    lines = file.readlines()

# Define a dictionary to map timestamps to corresponding letters
letter_mapping = {'C': 'C', 'B': 'B', 'A': 'A'}

# Initialize an empty list to store the formatted filenames
formatted_filenames = []

line_no=1
# Iterate through each line in the file
for line in lines:
    # Split the line by underscores
    parts = line.strip().split('_')

    # Extract relevant information
    name = parts[0]  # Name of the person
    timestamp = parts[1]  # Timestamps
    letter = parts[2]  # Letter
    extension = parts[3]  # File extension

    # Construct the formatted filename
    formatted_filename = f'duet/task{line_no}/timestamps_{name}_{letter_mapping[letter]}.txt'

    # Append the formatted filename to the list
    formatted_filenames.append(formatted_filename)

    line_no = line_no +1

print(formatted_filenames)

['duet/task1/timestamps_Prakul_C.txt', 'duet/task2/timestamps_Shiven_C.txt', 'duet/task3/timestamps_Poorvi_B.txt', 'duet/task4/timestamps_Shiven_A.txt']


In [59]:
song_ids = ['A','B','C','D']
file_paths = [f'duet/Ujjwal_files/timestamps_Ujjwal_{song_id}.txt' for song_id in song_ids]

for song_id, file_path, formatted_filename in zip(song_ids, file_paths, formatted_filenames):
    tapping_intervals_tapee = load_tapping_data(file_path)
    phase_angles_tapee = convert_ITIs_to_phase_angles(tapping_intervals_tapee)

    tapping_intervals_overlayee = load_tapping_data(formatted_filename)
    phase_angles_overlayee = convert_ITIs_to_phase_angles(tapping_intervals_overlayee)

    angle_diff = []
    for i in range(min(len(phase_angles_overlayee),len(phase_angles_tapee))):
        angle_diff.append(phase_angles_overlayee[i] - phase_angles_tapee[i])

    R_bar = calculate_R_bar(angle_diff)

    print(f"Stability measure: {song_id} : {R_bar}")

Stability measure: A : 0.8850906817891662
Stability measure: B : 0.7998102214641798
Stability measure: C : 0.6837774846822493
Stability measure: D : 0.8234038414017093
